## Greating EnzymeML document for PPCS for Sequential mechansim:

In [1]:
import glob, pickle, os
import numpy as np

from pyenzyme import (
    EnzymeMLDocument,
    Reactant, Complex, Vessel,
    Protein, EnzymeReaction,
    KineticModel, Measurement,
    Replicate
)

## Initial Set-up of EnzymeML document. Adding of vessels, proteins and reactants:

In [3]:
# Start of the EnzymeMLDocument
enzmldoc = EnzymeMLDocument(name="Phosphopantothenoylcysteine synthetase (PPCS)")

# Adding Vessel:
vessel = Vessel(name="96 well plate", volume=150, unit="ul")
vessel_id = enzmldoc.addVessel(vessel)

# Adding protein and reactants:
Ppan_id = enzmldoc.addReactant(Reactant(
    name = "4'-Phosphopantothenate", vessel_id = vessel_id, init_conc = 1,
    unit = "mM", constant = False, smiles = 'CC(C)(COP(O)(O)=O)[COOH](O)C(=O)NCCC(O)=O',
    chebi_id = 'CHEBI:15905',
))

ctp_id = enzmldoc.addReactant(Reactant(
    name = "Cytidine triphosphate", vessel_id = vessel_id, init_conc = 1,
    unit = "mM", constant = False, smiles = 'NC1CCN((COOH)2O(COH)(COP(O)(=O)OP(O)(=O)OP(O)(O)=O)(COOH)(O)(COH)2O)C(=O)n1',
    chebi_id = 'CHEBI:17677',
))

PPi_id = enzmldoc.addReactant(Reactant(
    name = "Inorganic Pyrophospahte", vessel_id = vessel_id, init_conc = 1,
    unit = "mM", constant = False, smiles = '[O-]P(=O)([O-])OP(=O)([O-])[O-]',
    chebi_id = 'CHEBI:18361',
))

In [4]:
cys_id = enzmldoc.addReactant(Reactant(
    name = "L-cysteine", vessel_id = vessel_id, init_conc = 1,
    unit = "mM", constant = False, smiles = 'N(COOH)(CS)C(O)=O',
    chebi_id = 'CHEBI:17561',
))

CMP_id = enzmldoc.addReactant(Reactant(
    name = "Cytidine monophosphate", vessel_id = vessel_id, init_conc = 1,
    unit = "mM", constant = False, smiles = 'c1cn(c(=O)nc1N)[COH]2[COOH]([COOH]([COH](O2)COP(=O)(O)O)O)O',
    chebi_id = 'CHEBI:20878',
))

Ppan_cys_id = enzmldoc.addReactant(Reactant(
    name = "4'-Phosphopantothenoylcysteine", vessel_id = vessel_id, init_conc = 1,
    unit = "mM", constant = False
))

    
coabc_id = enzmldoc.addProtein(Protein(
    name="CoaBC", sequence = "UNKNOWN", vessel_id=vessel_id, 
    organism="Staphylococcus aureus", init_conc=0.05, unit="mg / ml",
))

## Adding the reaction to the EnzymeML document: 

In [5]:
# Creating the reaction:

reaction1 = EnzymeReaction(temperature=25.0, temperature_unit="C", 
                           ph=7.8, name="Phosphopantothenoylcysteine synthetase", reversible=False)

reaction1.addEduct(Ppan_id, 1.0, enzmldoc)
reaction1.addEduct(ctp_id, 1.0, enzmldoc)
reaction1.addProduct(PPi_id, 1.0, enzmldoc)
reaction1.addModifier(coabc_id, 1.0, enzmldoc, False)
reaction1.addEduct(cys_id , 1.0, enzmldoc)
reaction1.addProduct(CMP_id , 1.0, enzmldoc)
reaction1.addProduct(Ppan_cys_id , 1.0, enzmldoc)



In [6]:
# Add the reactions to the  EnzymeML document

enzmldoc.addReactions([reaction1])

{'Phosphopantothenoylcysteine synthetase': 'r0'}

## Adding  measurements 

Create measurment for first condition at CTP = 1mM, PPan = 0.37 mM and Cys = 1mM

In [7]:
import numpy as np
import pandas as pd

# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1-CTP_1mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2-CTP_1mM.xlsx")
df3 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run3-CTP_1mM.xlsx")

# Create the Measurement object for one run at CTP = 1 mM
measurement_1 = Measurement(
    name="Run 1 - CTP = 1mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_1.addData(reactant_id=Ppan_id, init_conc=0.35, unit="mM")
measurement_1.addData(reactant_id=ctp_id, init_conc=1, unit="mM")
measurement_1.addData(reactant_id=cys_id, init_conc=1, unit="mM")
measurement_1.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

In [8]:
# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for CTP = 1 mM @ 0.37 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['B2'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for CTP = 1 mM @ 0.37 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['B4'].tolist()
)

repeat3 = Replicate(
    id="repeat3 for CTP = 1 mM @ 0.37 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df3['Time_min'].tolist(),
    data=df3['B2'].tolist()
)

# Add replicates to measurement
measurement_1.addReplicates(repeat1, enzmldoc)
measurement_1.addReplicates(repeat2, enzmldoc)
measurement_1.addReplicates(repeat3, enzmldoc)

# Add the measurement to the EnzymeML document
meas_1_id = enzmldoc.addMeasurement(measurement_1)

print("Measurement created for three repeats at CTP = 1 mM")

Measurement created for three repeats at CTP = 1 mM


Create measurment for second condition at CTP = 1 mM, Ppan = 0.18 mM and Cys = 1 mM

In [9]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1-CTP_1mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2-CTP_1mM.xlsx")
df3 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run3-CTP_1mM.xlsx")

# Create the Measurement object for one run at CTP = 1 mM
measurement_2 = Measurement(
    name="Run 2 - CTP = 1mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_2.addData(reactant_id=Ppan_id, init_conc=0.18, unit="mM")
measurement_2.addData(reactant_id=ctp_id, init_conc=1, unit="mM")
measurement_2.addData(reactant_id=cys_id, init_conc=1, unit="mM")
measurement_2.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for CTP = 1 mM @ 0.18 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['C2'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for CTP = 1 mM @ 0.18 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['C4'].tolist()
)

repeat3 = Replicate(
    id="repeat3 for CTP = 1 mM @ 0.18 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df3['Time_min'].tolist(),
    data=df3['C2'].tolist()
)

# Add replicates to measurement
measurement_2.addReplicates(repeat1, enzmldoc)
measurement_2.addReplicates(repeat2, enzmldoc)
measurement_2.addReplicates(repeat3, enzmldoc)

# Add the measurement to the EnzymeML document
meas_2_id = enzmldoc.addMeasurement(measurement_2)

print("Measurement created for three repeats at CTP = 1 mM, Ppan = 0.18 mM and Cys = 1mM")

Measurement created for three repeats at CTP = 1 mM, Ppan = 0.18 mM and Cys = 1mM


Create measurment for second condition at CTP = 1 mM, Ppan = 0.093 mM and Cys = 1 mM

In [10]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1-CTP_1mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2-CTP_1mM.xlsx")
df3 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run3-CTP_1mM.xlsx")

# Create the Measurement object for one run at CTP = 1 mM
measurement_3 = Measurement(
    name="Run 3 - CTP = 1mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_3.addData(reactant_id=Ppan_id, init_conc=0.093, unit="mM")
measurement_3.addData(reactant_id=ctp_id, init_conc=1, unit="mM")
measurement_3.addData(reactant_id=cys_id, init_conc=1, unit="mM")
measurement_3.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for CTP = 1 mM @ 0.093 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['D2'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for CTP = 1 mM @ 0.093 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['D4'].tolist()
)

repeat3 = Replicate(
    id="repeat3 for CTP = 1 mM @ 0.093 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df3['Time_min'].tolist(),
    data=df3['D2'].tolist()
)

# Add replicates to measurement
measurement_3.addReplicates(repeat1, enzmldoc)
measurement_3.addReplicates(repeat2, enzmldoc)
measurement_3.addReplicates(repeat3, enzmldoc)

# Add the measurement to the EnzymeML document
meas_3_id = enzmldoc.addMeasurement(measurement_3)

print("Measurement created for three repeats at CTP = 1 mM, Ppan = 0.093 mM and Cys = 1mM")

Measurement created for three repeats at CTP = 1 mM, Ppan = 0.093 mM and Cys = 1mM


Create measurment for second condition at CTP = 1 mM, Ppan = 0.046 mM and Cys = 1 mM

In [11]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1-CTP_1mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2-CTP_1mM.xlsx")
df3 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run3-CTP_1mM.xlsx")

# Create the Measurement object for one run at CTP = 1 mM
measurement_4 = Measurement(
    name="Run 4 - CTP = 1mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_4.addData(reactant_id=Ppan_id, init_conc=0.046, unit="mM")
measurement_4.addData(reactant_id=ctp_id, init_conc=1, unit="mM")
measurement_4.addData(reactant_id=cys_id, init_conc=1, unit="mM")
measurement_4.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for CTP = 1 mM @ 0.046mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['E2'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for CTP = 1 mM @ 0.046 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['E4'].tolist()
)

repeat3 = Replicate(
    id="repeat3 for CTP = 1 mM @ 0.046 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df3['Time_min'].tolist(),
    data=df3['E2'].tolist()
)

# Add replicates to measurement
measurement_4.addReplicates(repeat1, enzmldoc)
measurement_4.addReplicates(repeat2, enzmldoc)
measurement_4.addReplicates(repeat3, enzmldoc)

# Add the measurement to the EnzymeML document
meas_4_id = enzmldoc.addMeasurement(measurement_4)

print("Measurement created for three repeats at CTP = 1 mM, Ppan = 0.046 mM and Cys = 1mM")

Measurement created for three repeats at CTP = 1 mM, Ppan = 0.046 mM and Cys = 1mM


Create measurment for second condition at CTP = 1 mM, Ppan = 0.023 mM and Cys = 1 mM

In [12]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1-CTP_1mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2-CTP_1mM.xlsx")
df3 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run3-CTP_1mM.xlsx")

# Create the Measurement object for one run at CTP = 1 mM
measurement_5 = Measurement(
    name="Run 5 - CTP = 1mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_5.addData(reactant_id=Ppan_id, init_conc=0.023, unit="mM")
measurement_5.addData(reactant_id=ctp_id, init_conc=1, unit="mM")
measurement_5.addData(reactant_id=cys_id, init_conc=1, unit="mM")
measurement_5.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for CTP = 1 mM @ 0.023mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['F2'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for CTP = 1 mM @ 0.023 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['F4'].tolist()
)

repeat3 = Replicate(
    id="repeat3 for CTP = 1 mM @ 0.023 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df3['Time_min'].tolist(),
    data=df3['F2'].tolist()
)

# Add replicates to measurement
measurement_5.addReplicates(repeat1, enzmldoc)
measurement_5.addReplicates(repeat2, enzmldoc)
measurement_5.addReplicates(repeat3, enzmldoc)

# Add the measurement to the EnzymeML document
meas_5_id = enzmldoc.addMeasurement(measurement_5)

print("Measurement created for three repeats at CTP = 1 mM, Ppan = 0.023 mM and Cys = 1mM")

Measurement created for three repeats at CTP = 1 mM, Ppan = 0.023 mM and Cys = 1mM


New concnetration for CTP: Create measurment for CTP = 0.5 mM, PPan = 0.37 mM and Cys = 1mM

In [13]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1-CTP_0_5mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2-CTP_0_5mM.xlsx")
df3 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run3-CTP_0_5mM.xlsx")

# Create the Measurement object for one run at CTP = 1 mM
measurement_6 = Measurement(
    name="Run 6 - CTP = 0.5mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_6.addData(reactant_id=Ppan_id, init_conc=0.37, unit="mM")
measurement_6.addData(reactant_id=ctp_id, init_conc=0.5, unit="mM")
measurement_6.addData(reactant_id=cys_id, init_conc=1, unit="mM")
measurement_6.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for CTP = 1 mM @ 0.37 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['B6'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for CTP = 1 mM @ 0.37 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['B8'].tolist()
)

repeat3 = Replicate(
    id="repeat3 for CTP = 1 mM @ 0.37 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df3['Time_min'].tolist(),
    data=df3['B8'].tolist()
)

# Add replicates to measurement
measurement_6.addReplicates(repeat1, enzmldoc)
measurement_6.addReplicates(repeat2, enzmldoc)
measurement_6.addReplicates(repeat3, enzmldoc)

# Add the measurement to the EnzymeML document
meas_6_id = enzmldoc.addMeasurement(measurement_6)

print("Measurement created for three repeats at CTP = 0.5 mM, Ppan = 0.37 mM and Cys = 1mM")


Measurement created for three repeats at CTP = 0.5 mM, Ppan = 0.37 mM and Cys = 1mM


 Create measurment for CTP = 0.5 mM, PPan = 0.18 mM and Cys = 1mM

In [14]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1-CTP_0_5mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2-CTP_0_5mM.xlsx")
df3 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run3-CTP_0_5mM.xlsx")

# Create the Measurement object for one run at CTP = 1 mM
measurement_7 = Measurement(
    name="Run 7 - CTP = 0.5mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_7.addData(reactant_id=Ppan_id, init_conc=0.18, unit="mM")
measurement_7.addData(reactant_id=ctp_id, init_conc=0.5, unit="mM")
measurement_7.addData(reactant_id=cys_id, init_conc=1, unit="mM")
measurement_7.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for CTP = 1 mM @ 0.18 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['C6'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for CTP = 1 mM @ 0.18 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['C8'].tolist()
)

repeat3 = Replicate(
    id="repeat3 for CTP = 1 mM @ 0.18 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df3['Time_min'].tolist(),
    data=df3['C8'].tolist()
)

# Add replicates to measurement
measurement_7.addReplicates(repeat1, enzmldoc)
measurement_7.addReplicates(repeat2, enzmldoc)
measurement_7.addReplicates(repeat3, enzmldoc)

# Add the measurement to the EnzymeML document
meas_7_id = enzmldoc.addMeasurement(measurement_7)

print("Measurement created for three repeats at CTP = 0.5 mM, Ppan = 0.18 mM and Cys = 1mM")


Measurement created for three repeats at CTP = 0.5 mM, Ppan = 0.18 mM and Cys = 1mM


Create measurment for CTP = 0.5 mM, PPan = 0.093 mM and Cys = 1mM

In [15]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1-CTP_0_5mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2-CTP_0_5mM.xlsx")
df3 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run3-CTP_0_5mM.xlsx")

# Create the Measurement object for one run at CTP = 1 mM
measurement_8 = Measurement(
    name="Run 8 - CTP = 0.5mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_8.addData(reactant_id=Ppan_id, init_conc=0.093, unit="mM")
measurement_8.addData(reactant_id=ctp_id, init_conc=0.5, unit="mM")
measurement_8.addData(reactant_id=cys_id, init_conc=1, unit="mM")
measurement_8.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for CTP = 1 mM @ 0.093 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['D6'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for CTP = 1 mM @ 0.093 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['D8'].tolist()
)

repeat3 = Replicate(
    id="repeat3 for CTP = 1 mM @ 0.093 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df3['Time_min'].tolist(),
    data=df3['D8'].tolist()
)

# Add replicates to measurement
measurement_8.addReplicates(repeat1, enzmldoc)
measurement_8.addReplicates(repeat2, enzmldoc)
measurement_8.addReplicates(repeat3, enzmldoc)

# Add the measurement to the EnzymeML document
meas_8_id = enzmldoc.addMeasurement(measurement_8)

print("Measurement created for three repeats at CTP = 0.5 mM, Ppan = 0.093 mM and Cys = 1mM")


Measurement created for three repeats at CTP = 0.5 mM, Ppan = 0.093 mM and Cys = 1mM


Create measurment for CTP = 0.5 mM, PPan = 0.046 mM and Cys = 1mM

In [16]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1-CTP_0_5mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2-CTP_0_5mM.xlsx")
df3 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run3-CTP_0_5mM.xlsx")

# Create the Measurement object for one run at CTP = 1 mM
measurement_9 = Measurement(
    name="Run 9 - CTP = 0.5mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_9.addData(reactant_id=Ppan_id, init_conc=0.046, unit="mM")
measurement_9.addData(reactant_id=ctp_id, init_conc=0.5, unit="mM")
measurement_9.addData(reactant_id=cys_id, init_conc=1, unit="mM")
measurement_9.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for CTP = 1 mM @ 0.046 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['E6'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for CTP = 1 mM @ 0.046 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['E8'].tolist()
)

repeat3 = Replicate(
    id="repeat3 for CTP = 1 mM @ 0.046 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df3['Time_min'].tolist(),
    data=df3['E8'].tolist()
)

# Add replicates to measurement
measurement_9.addReplicates(repeat1, enzmldoc)
measurement_9.addReplicates(repeat2, enzmldoc)
measurement_9.addReplicates(repeat3, enzmldoc)

# Add the measurement to the EnzymeML document
meas_9_id = enzmldoc.addMeasurement(measurement_9)

print("Measurement created for three repeats at CTP = 0.5 mM, Ppan = 0.046 mM and Cys = 1mM")


Measurement created for three repeats at CTP = 0.5 mM, Ppan = 0.046 mM and Cys = 1mM


Create measurment for CTP = 0.5 mM, PPan = 0.023 mM and Cys = 1mM

In [17]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1-CTP_0_5mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2-CTP_0_5mM.xlsx")
df3 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run3-CTP_0_5mM.xlsx")

# Create the Measurement object for one run at CTP = 1 mM
measurement_10 = Measurement(
    name="Run 10 - CTP = 0.5mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_10.addData(reactant_id=Ppan_id, init_conc=0.023, unit="mM")
measurement_10.addData(reactant_id=ctp_id, init_conc=0.5, unit="mM")
measurement_10.addData(reactant_id=cys_id, init_conc=1, unit="mM")
measurement_10.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for CTP = 1 mM @ 0.023 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['F6'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for CTP = 1 mM @ 0.023 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['F8'].tolist()
)

repeat3 = Replicate(
    id="repeat3 for CTP = 1 mM @ 0.023 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df3['Time_min'].tolist(),
    data=df3['F8'].tolist()
)

# Add replicates to measurement
measurement_10.addReplicates(repeat1, enzmldoc)
measurement_10.addReplicates(repeat2, enzmldoc)
measurement_10.addReplicates(repeat3, enzmldoc)

# Add the measurement to the EnzymeML document
meas_10_id = enzmldoc.addMeasurement(measurement_10)

print("Measurement created for three repeats at CTP = 0.5 mM, Ppan = 0.023 mM and Cys = 1mM")


Measurement created for three repeats at CTP = 0.5 mM, Ppan = 0.023 mM and Cys = 1mM


Create new measurement for CTP: Create measurement for CTP = 0.25 mM, PPan = 0.37 mM and Cys = 1mM

In [18]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1-CTP_0_25mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2-CTP_0_25mM.xlsx")
df3 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run3-CTP_0_25mM.xlsx")

# Create the Measurement object for one run at CTP = 1 mM
measurement_11 = Measurement(
    name="Run 11 - CTP = 0.25mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_11.addData(reactant_id=Ppan_id, init_conc=0.37, unit="mM")
measurement_11.addData(reactant_id=ctp_id, init_conc=0.25, unit="mM")
measurement_11.addData(reactant_id=cys_id, init_conc=1, unit="mM")
measurement_11.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for CTP = 0.25 mM @ 0.37 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['B10'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for CTP = 0.25 mM @ 0.37 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['B12'].tolist()
)

repeat3 = Replicate(
    id="repeat3 for CTP = 0.25 mM @ 0.37 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df3['Time_min'].tolist(),
    data=df3['B10'].tolist()
)

# Add replicates to measurement
measurement_11.addReplicates(repeat1, enzmldoc)
measurement_11.addReplicates(repeat2, enzmldoc)
measurement_11.addReplicates(repeat3, enzmldoc)

# Add the measurement to the EnzymeML document
meas_11_id = enzmldoc.addMeasurement(measurement_11)

print("Measurement created for three repeats at CTP = 0.25 mM, Ppan = 0.37 mM and Cys = 1mM")


Measurement created for three repeats at CTP = 0.25 mM, Ppan = 0.37 mM and Cys = 1mM


 Create measurement for CTP = 0.25 mM, PPan = 0.18 mM and Cys = 1mM

In [19]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1-CTP_0_25mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2-CTP_0_25mM.xlsx")
df3 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run3-CTP_0_25mM.xlsx")

# Create the Measurement object for one run at CTP = 1 mM
measurement_12 = Measurement(
    name="Run 12 - CTP = 0.25mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_12.addData(reactant_id=Ppan_id, init_conc=0.18, unit="mM")
measurement_12.addData(reactant_id=ctp_id, init_conc=0.25, unit="mM")
measurement_12.addData(reactant_id=cys_id, init_conc=1, unit="mM")
measurement_12.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for CTP = 0.25 mM @ 0.18 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['C10'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for CTP = 0.25 mM @ 0.18 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['C12'].tolist()
)

repeat3 = Replicate(
    id="repeat3 for CTP = 0.25 mM @ 0.18 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df3['Time_min'].tolist(),
    data=df3['C10'].tolist()
)

# Add replicates to measurement
measurement_12.addReplicates(repeat1, enzmldoc)
measurement_12.addReplicates(repeat2, enzmldoc)
measurement_12.addReplicates(repeat3, enzmldoc)

# Add the measurement to the EnzymeML document
meas_12_id = enzmldoc.addMeasurement(measurement_12)

print("Measurement created for three repeats at CTP = 0.25 mM, Ppan = 0.18 mM and Cys = 1mM")


Measurement created for three repeats at CTP = 0.25 mM, Ppan = 0.18 mM and Cys = 1mM


 Create measurement for CTP = 0.25 mM, PPan = 0.093 mM and Cys = 1mM

In [20]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1-CTP_0_25mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2-CTP_0_25mM.xlsx")
df3 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run3-CTP_0_25mM.xlsx")

# Create the Measurement object for one run at CTP = 1 mM
measurement_13 = Measurement(
    name="Run 13 - CTP = 0.25mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_13.addData(reactant_id=Ppan_id, init_conc=0.093, unit="mM")
measurement_13.addData(reactant_id=ctp_id, init_conc=0.25, unit="mM")
measurement_13.addData(reactant_id=cys_id, init_conc=1, unit="mM")
measurement_13.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for CTP = 0.25 mM @ 0.093 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['D10'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for CTP = 0.25 mM @ 0.093 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['D12'].tolist()
)

repeat3 = Replicate(
    id="repeat3 for CTP = 0.25 mM @ 0.093 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df3['Time_min'].tolist(),
    data=df3['D10'].tolist()
)

# Add replicates to measurement
measurement_13.addReplicates(repeat1, enzmldoc)
measurement_13.addReplicates(repeat2, enzmldoc)
measurement_13.addReplicates(repeat3, enzmldoc)

# Add the measurement to the EnzymeML document
meas_13_id = enzmldoc.addMeasurement(measurement_13)

print("Measurement created for three repeats at CTP = 0.25 mM, Ppan = 0.093 mM and Cys = 1mM")


Measurement created for three repeats at CTP = 0.25 mM, Ppan = 0.093 mM and Cys = 1mM


 Create measurement for CTP = 0.25 mM, PPan = 0.046 mM and Cys = 1mM

In [21]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1-CTP_0_25mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2-CTP_0_25mM.xlsx")
df3 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run3-CTP_0_25mM.xlsx")

# Create the Measurement object for one run at CTP = 0.25 mM
measurement_14 = Measurement(
    name="Run 14 - CTP = 0.25mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_14.addData(reactant_id=Ppan_id, init_conc=0.046, unit="mM")
measurement_14.addData(reactant_id=ctp_id, init_conc=0.25, unit="mM")
measurement_14.addData(reactant_id=cys_id, init_conc=1, unit="mM")
measurement_14.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for CTP = 0.25 mM @ 0.046 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['E10'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for CTP = 0.25 mM @ 0.046 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['E12'].tolist()
)

repeat3 = Replicate(
    id="repeat3 for CTP = 0.25 mM @ 0.046 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df3['Time_min'].tolist(),
    data=df3['E10'].tolist()
)

# Add replicates to measurement
measurement_14.addReplicates(repeat1, enzmldoc)
measurement_14.addReplicates(repeat2, enzmldoc)
measurement_14.addReplicates(repeat3, enzmldoc)

# Add the measurement to the EnzymeML document
meas_14_id = enzmldoc.addMeasurement(measurement_14)

print("Measurement created for three repeats at CTP = 0.25 mM, Ppan = 0.046 mM and Cys = 1mM")


Measurement created for three repeats at CTP = 0.25 mM, Ppan = 0.046 mM and Cys = 1mM


Create measurement for CTP = 0.25 mM, PPan = 0.023 mM and Cys = 1mM

In [22]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1-CTP_0_25mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2-CTP_0_25mM.xlsx")
df3 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run3-CTP_0_25mM.xlsx")

# Create the Measurement object for one run at CTP = 0.25 mM
measurement_15 = Measurement(
    name="Run 15 - CTP = 0.25mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_15.addData(reactant_id=Ppan_id, init_conc=0.023, unit="mM")
measurement_15.addData(reactant_id=ctp_id, init_conc=0.25, unit="mM")
measurement_15.addData(reactant_id=cys_id, init_conc=1, unit="mM")
measurement_15.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for CTP = 0.25 mM @ 0.023 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['F10'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for CTP = 0.25 mM @ 0.023 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['F12'].tolist()
)

repeat3 = Replicate(
    id="repeat3 for CTP = 0.25 mM @ 0.023 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df3['Time_min'].tolist(),
    data=df3['F10'].tolist()
)

# Add replicates to measurement
measurement_15.addReplicates(repeat1, enzmldoc)
measurement_15.addReplicates(repeat2, enzmldoc)
measurement_15.addReplicates(repeat3, enzmldoc)

# Add the measurement to the EnzymeML document
meas_15_id = enzmldoc.addMeasurement(measurement_15)

print("Measurement created for three repeats at CTP = 0.25 mM, Ppan = 0.023 mM and Cys = 1mM")

Measurement created for three repeats at CTP = 0.25 mM, Ppan = 0.023 mM and Cys = 1mM


Create new  measurement for CTP:  Create measurement for CTP = 0.125 mM, PPan = 0.37 mM and Cys = 1mM

In [23]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1-CTP_0_125mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2-CTP_0_125mM.xlsx")
df3 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run3-CTP_0_125mM.xlsx")

# Create the Measurement object for one run at CTP = 0.125 mM
measurement_16 = Measurement(
    name="Run 16 - CTP = 0.125mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_16.addData(reactant_id=Ppan_id, init_conc=0.37, unit="mM")
measurement_16.addData(reactant_id=ctp_id, init_conc=0.125, unit="mM")
measurement_16.addData(reactant_id=cys_id, init_conc=1, unit="mM")
measurement_16.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for CTP = 0.125 mM @ 0.37 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['B12'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for CTP = 0.125 mM @ 0.37 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['B2'].tolist()
)

repeat3 = Replicate(
    id="repeat3 for CTP = 0.125 mM @ 0.37 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df3['Time_min'].tolist(),
    data=df3['B4'].tolist()
)

# Add replicates to measurement
measurement_16.addReplicates(repeat1, enzmldoc)
measurement_16.addReplicates(repeat2, enzmldoc)
measurement_16.addReplicates(repeat3, enzmldoc)

# Add the measurement to the EnzymeML document
meas_16_id = enzmldoc.addMeasurement(measurement_16)

print("Measurement created for three repeats at CTP = 0.125 mM, Ppan = 0.37 mM and Cys = 1mM")

Measurement created for three repeats at CTP = 0.125 mM, Ppan = 0.37 mM and Cys = 1mM


 Create measurement for CTP = 0.125 mM, PPan = 0.18 mM and Cys = 1mM

In [24]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1-CTP_0_125mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2-CTP_0_125mM.xlsx")
df3 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run3-CTP_0_125mM.xlsx")

# Create the Measurement object for one run at CTP = 0.125 mM
measurement_17 = Measurement(
    name="Run 17 - CTP = 0.125mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_17.addData(reactant_id=Ppan_id, init_conc=0.18, unit="mM")
measurement_17.addData(reactant_id=ctp_id, init_conc=0.125, unit="mM")
measurement_17.addData(reactant_id=cys_id, init_conc=1, unit="mM")
measurement_17.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for CTP = 0.125 mM @ 0.18 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['C12'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for CTP = 0.125 mM @ 0.18 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['C2'].tolist()
)

repeat3 = Replicate(
    id="repeat3 for CTP = 0.125 mM @ 0.18 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df3['Time_min'].tolist(),
    data=df3['C4'].tolist()
)

# Add replicates to measurement
measurement_17.addReplicates(repeat1, enzmldoc)
measurement_17.addReplicates(repeat2, enzmldoc)
measurement_17.addReplicates(repeat3, enzmldoc)

# Add the measurement to the EnzymeML document
meas_17_id = enzmldoc.addMeasurement(measurement_17)

print("Measurement created for three repeats at CTP = 0.125 mM, Ppan = 0.18 mM and Cys = 1mM")

Measurement created for three repeats at CTP = 0.125 mM, Ppan = 0.18 mM and Cys = 1mM


Create measurement for CTP = 0.125 mM, PPan = 0.093 mM and Cys = 1mM

In [25]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1-CTP_0_125mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2-CTP_0_125mM.xlsx")
df3 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run3-CTP_0_125mM.xlsx")

# Create the Measurement object for one run at CTP = 0.125 mM
measurement_18 = Measurement(
    name="Run 18 - CTP = 0.125mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_18.addData(reactant_id=Ppan_id, init_conc=0.093, unit="mM")
measurement_18.addData(reactant_id=ctp_id, init_conc=0.125, unit="mM")
measurement_18.addData(reactant_id=cys_id, init_conc=1, unit="mM")
measurement_18.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for CTP = 0.125 mM @ 0.093 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['D12'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for CTP = 0.125 mM @ 0.093 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['D2'].tolist()
)

repeat3 = Replicate(
    id="repeat3 for CTP = 0.125 mM @ 0.093 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df3['Time_min'].tolist(),
    data=df3['D4'].tolist()
)

# Add replicates to measurement
measurement_18.addReplicates(repeat1, enzmldoc)
measurement_18.addReplicates(repeat2, enzmldoc)
measurement_18.addReplicates(repeat3, enzmldoc)

# Add the measurement to the EnzymeML document
meas_18_id = enzmldoc.addMeasurement(measurement_18)

print("Measurement created for three repeats at CTP = 0.125 mM, Ppan = 0.093 mM and Cys = 1mM")

Measurement created for three repeats at CTP = 0.125 mM, Ppan = 0.093 mM and Cys = 1mM


Create measurement for CTP = 0.125 mM, PPan = 0.046 mM and Cys = 1mM

In [26]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1-CTP_0_125mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2-CTP_0_125mM.xlsx")
df3 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run3-CTP_0_125mM.xlsx")

# Create the Measurement object for one run at CTP = 0.125 mM
measurement_19 = Measurement(
    name="Run 19 - CTP = 0.125mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_19.addData(reactant_id=Ppan_id, init_conc=0.046, unit="mM")
measurement_19.addData(reactant_id=ctp_id, init_conc=0.125, unit="mM")
measurement_19.addData(reactant_id=cys_id, init_conc=1, unit="mM")
measurement_19.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for CTP = 0.125 mM @ 0.046 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['E12'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for CTP = 0.125 mM @ 0.046 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['E2'].tolist()
)

repeat3 = Replicate(
    id="repeat3 for CTP = 0.125 mM @ 0.046 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df3['Time_min'].tolist(),
    data=df3['E4'].tolist()
)

# Add replicates to measurement
measurement_19.addReplicates(repeat1, enzmldoc)
measurement_19.addReplicates(repeat2, enzmldoc)
measurement_19.addReplicates(repeat3, enzmldoc)

# Add the measurement to the EnzymeML document
meas_19_id = enzmldoc.addMeasurement(measurement_19)

print("Measurement created for three repeats at CTP = 0.125 mM, Ppan = 0.046 mM and Cys = 1mM")

Measurement created for three repeats at CTP = 0.125 mM, Ppan = 0.046 mM and Cys = 1mM


Create measurement for CTP = 0.125 mM, PPan = 0.023 mM and Cys = 1mM

In [27]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1-CTP_0_125mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2-CTP_0_125mM.xlsx")
df3 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run3-CTP_0_125mM.xlsx")

# Create the Measurement object for one run at CTP = 0.125 mM
measurement_20 = Measurement(
    name="Run 20 - CTP = 0.125mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_20.addData(reactant_id=Ppan_id, init_conc=0.023, unit="mM")
measurement_20.addData(reactant_id=ctp_id, init_conc=0.125, unit="mM")
measurement_20.addData(reactant_id=cys_id, init_conc=1, unit="mM")
measurement_20.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for CTP = 0.125 mM @ 0.023 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['F12'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for CTP = 0.125 mM @ 0.023 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['F2'].tolist()
)

repeat3 = Replicate(
    id="repeat3 for CTP = 0.125 mM @ 0.023 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df3['Time_min'].tolist(),
    data=df3['F4'].tolist()
)

# Add replicates to measurement
measurement_20.addReplicates(repeat1, enzmldoc)
measurement_20.addReplicates(repeat2, enzmldoc)
measurement_20.addReplicates(repeat3, enzmldoc)

# Add the measurement to the EnzymeML document
meas_20_id = enzmldoc.addMeasurement(measurement_20)

print("Measurement created for three repeats at CTP = 0.125 mM, Ppan = 0.023 mM and Cys = 1mM")

Measurement created for three repeats at CTP = 0.125 mM, Ppan = 0.023 mM and Cys = 1mM


Create new measurement for CTP: measurement for CTP = 0.0625 mM, PPan = 0.37 mM and Cys = 1mM

In [28]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1-CTP_0_0625.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2-CTP_0_0625.xlsx")

# Create the Measurement object for one run at CTP = 0.125 mM
measurement_21 = Measurement(
    name="Run 21 - CTP = 0.0625mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_21.addData(reactant_id=Ppan_id, init_conc=0.37, unit="mM")
measurement_21.addData(reactant_id=ctp_id, init_conc=0.0625, unit="mM")
measurement_21.addData(reactant_id=cys_id, init_conc=1, unit="mM")
measurement_21.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for CTP = 0.0625 mM @ 0.37 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['B6'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for CTP = 0.0625 mM @ 0.37 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['B8'].tolist()
)

# Add replicates to measurement
measurement_21.addReplicates(repeat1, enzmldoc)
measurement_21.addReplicates(repeat2, enzmldoc)

# Add the measurement to the EnzymeML document
meas_21_id = enzmldoc.addMeasurement(measurement_21)

print("Measurement created for two repeats at CTP = 0.0625 mM, Ppan = 0.37 mM and Cys = 1mM")

Measurement created for two repeats at CTP = 0.0625 mM, Ppan = 0.37 mM and Cys = 1mM


Create measurement for CTP = 0.0625 mM, PPan = 0.18 mM and Cys = 1mM

In [29]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1-CTP_0_0625.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2-CTP_0_0625.xlsx")

# Create the Measurement object for one run at CTP = 0.0625 mM
measurement_22 = Measurement(
    name="Run 22 - CTP = 0.0625mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_22.addData(reactant_id=Ppan_id, init_conc=0.18, unit="mM")
measurement_22.addData(reactant_id=ctp_id, init_conc=0.0625, unit="mM")
measurement_22.addData(reactant_id=cys_id, init_conc=1, unit="mM")
measurement_22.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for CTP = 0.0625 mM @ 0.18 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['C6'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for CTP = 0.0625 mM @ 0.18 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['C8'].tolist()
)

# Add replicates to measurement
measurement_22.addReplicates(repeat1, enzmldoc)
measurement_22.addReplicates(repeat2, enzmldoc)

# Add the measurement to the EnzymeML document
meas_22_id = enzmldoc.addMeasurement(measurement_22)

print("Measurement created for two repeats at CTP = 0.0625 mM, Ppan = 0.18 mM and Cys = 1mM")

Measurement created for two repeats at CTP = 0.0625 mM, Ppan = 0.18 mM and Cys = 1mM


Create measurement for CTP = 0.0625 mM, PPan = 0.093 mM and Cys = 1mM

In [30]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1-CTP_0_0625.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2-CTP_0_0625.xlsx")

# Create the Measurement object for one run at CTP = 0.0625 mM
measurement_23 = Measurement(
    name="Run 23 - CTP = 0.0625mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_23.addData(reactant_id=Ppan_id, init_conc=0.093, unit="mM")
measurement_23.addData(reactant_id=ctp_id, init_conc=0.0625, unit="mM")
measurement_23.addData(reactant_id=cys_id, init_conc=1, unit="mM")
measurement_23.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for CTP = 0.0625 mM @ 0.093 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['D6'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for CTP = 0.0625 mM @ 0.093 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['D8'].tolist()
)

# Add replicates to measurement
measurement_23.addReplicates(repeat1, enzmldoc)
measurement_23.addReplicates(repeat2, enzmldoc)

# Add the measurement to the EnzymeML document
meas_23_id = enzmldoc.addMeasurement(measurement_23)

print("Measurement created for two repeats at CTP = 0.0625 mM, Ppan = 0.093 mM and Cys = 1mM")

Measurement created for two repeats at CTP = 0.0625 mM, Ppan = 0.093 mM and Cys = 1mM


Create measurement for CTP = 0.0625 mM, PPan = 0.046 mM and Cys = 1mM

In [31]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1-CTP_0_0625.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2-CTP_0_0625.xlsx")

# Create the Measurement object for one run at CTP = 0.0625 mM
measurement_24 = Measurement(
    name="Run 24 - CTP = 0.0625mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_24.addData(reactant_id=Ppan_id, init_conc=0.046, unit="mM")
measurement_24.addData(reactant_id=ctp_id, init_conc=0.0625, unit="mM")
measurement_24.addData(reactant_id=cys_id, init_conc=1, unit="mM")
measurement_24.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for CTP = 0.0625 mM @ 0.046 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['E6'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for CTP = 0.0625 mM @ 0.046 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['E8'].tolist()
)

# Add replicates to measurement
measurement_24.addReplicates(repeat1, enzmldoc)
measurement_24.addReplicates(repeat2, enzmldoc)

# Add the measurement to the EnzymeML document
meas_24_id = enzmldoc.addMeasurement(measurement_24)

print("Measurement created for two repeats at CTP = 0.0625 mM, Ppan = 0.046 mM and Cys = 1mM")

Measurement created for two repeats at CTP = 0.0625 mM, Ppan = 0.046 mM and Cys = 1mM


Create measurement for CTP = 0.0625 mM, PPan = 0.023 mM and Cys = 1mM

In [32]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1-CTP_0_0625.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2-CTP_0_0625.xlsx")

# Create the Measurement object for one run at CTP = 0.0625 mM
measurement_25 = Measurement(
    name="Run 25 - CTP = 0.0625mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_25.addData(reactant_id=Ppan_id, init_conc=0.023, unit="mM")
measurement_25.addData(reactant_id=ctp_id, init_conc=0.0625, unit="mM")
measurement_25.addData(reactant_id=cys_id, init_conc=1, unit="mM")
measurement_25.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for CTP = 0.0625 mM @ 0.023 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['F6'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for CTP = 0.0625 mM @ 0.023 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['F8'].tolist()
)

# Add replicates to measurement
measurement_25.addReplicates(repeat1, enzmldoc)
measurement_25.addReplicates(repeat2, enzmldoc)

# Add the measurement to the EnzymeML document
meas_25_id = enzmldoc.addMeasurement(measurement_25)

print("Measurement created for two repeats at CTP = 0.0625 mM, Ppan = 0.023 mM and Cys = 1mM")

Measurement created for two repeats at CTP = 0.0625 mM, Ppan = 0.023 mM and Cys = 1mM


Create NEW measurement : Create measurement for CTP = 0.0312 mM, PPan = 0.37 mM and Cys = 1mM

In [33]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1-CTP_0_0312.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2-CTP_0_0312.xlsx")

# Create the Measurement object for one run at CTP = 0.0312 mM
measurement_26 = Measurement(
    name="Run 26 - CTP = 0.0312 mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_26.addData(reactant_id=Ppan_id, init_conc=0.37, unit="mM")
measurement_26.addData(reactant_id=ctp_id, init_conc=0.0312, unit="mM")
measurement_26.addData(reactant_id=cys_id, init_conc=1, unit="mM")
measurement_26.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for CTP = 0.0312mM @ 0.37 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['B10'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for CTP = 0.0312 mM @ 0.37 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['B12'].tolist()
)

# Add replicates to measurement
measurement_26.addReplicates(repeat1, enzmldoc)
measurement_26.addReplicates(repeat2, enzmldoc)

# Add the measurement to the EnzymeML document
meas_26_id = enzmldoc.addMeasurement(measurement_26)

print("Measurement created for two repeats at CTP = 0.0312 mM, Ppan = 0.37 mM and Cys = 1mM")

Measurement created for two repeats at CTP = 0.0312 mM, Ppan = 0.37 mM and Cys = 1mM


 Create measurement for CTP = 0.0312 mM, PPan = 0.18 mM and Cys = 1mM

In [34]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1-CTP_0_0312.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2-CTP_0_0312.xlsx")

# Create the Measurement object for one run at CTP = 0.0312 mM
measurement_27 = Measurement(
    name="Run 27 - CTP = 0.0312 mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_27.addData(reactant_id=Ppan_id, init_conc=0.18, unit="mM")
measurement_27.addData(reactant_id=ctp_id, init_conc=0.0312, unit="mM")
measurement_27.addData(reactant_id=cys_id, init_conc=1, unit="mM")
measurement_27.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for CTP = 0.0312mM @ 0.18 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['C10'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for CTP = 0.0312 mM @ 0.18 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['C12'].tolist()
)

# Add replicates to measurement
measurement_27.addReplicates(repeat1, enzmldoc)
measurement_27.addReplicates(repeat2, enzmldoc)

# Add the measurement to the EnzymeML document
meas_27_id = enzmldoc.addMeasurement(measurement_27)

print("Measurement created for two repeats at CTP = 0.0312 mM, Ppan = 0.18 mM and Cys = 1mM")

Measurement created for two repeats at CTP = 0.0312 mM, Ppan = 0.18 mM and Cys = 1mM


Create measurement for CTP = 0.0312 mM, PPan = 0.093 mM and Cys = 1mM

In [35]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1-CTP_0_0312.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2-CTP_0_0312.xlsx")

# Create the Measurement object for one run at CTP = 0.0312 mM
measurement_28 = Measurement(
    name="Run 28 - CTP = 0.0312 mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_28.addData(reactant_id=Ppan_id, init_conc=0.093, unit="mM")
measurement_28.addData(reactant_id=ctp_id, init_conc=0.0312, unit="mM")
measurement_28.addData(reactant_id=cys_id, init_conc=1, unit="mM")
measurement_28.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for CTP = 0.0312mM @ 0.093 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['D10'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for CTP = 0.0312 mM @ 0.093 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['D12'].tolist()
)

# Add replicates to measurement
measurement_28.addReplicates(repeat1, enzmldoc)
measurement_28.addReplicates(repeat2, enzmldoc)

# Add the measurement to the EnzymeML document
meas_28_id = enzmldoc.addMeasurement(measurement_28)

print("Measurement created for two repeats at CTP = 0.0312 mM, Ppan = 0.093 mM and Cys = 1mM")

Measurement created for two repeats at CTP = 0.0312 mM, Ppan = 0.093 mM and Cys = 1mM


Create measurement for CTP = 0.0312 mM, PPan = 0.046 mM and Cys = 1mM

In [36]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1-CTP_0_0312.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2-CTP_0_0312.xlsx")

# Create the Measurement object for one run at CTP = 0.0312 mM
measurement_29 = Measurement(
    name="Run 29 - CTP = 0.0312 mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_29.addData(reactant_id=Ppan_id, init_conc=0.046, unit="mM")
measurement_29.addData(reactant_id=ctp_id, init_conc=0.0312, unit="mM")
measurement_29.addData(reactant_id=cys_id, init_conc=1, unit="mM")
measurement_29.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for CTP = 0.0312mM @ 0.046 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['E10'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for CTP = 0.0312 mM @ 0.046 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['E12'].tolist()
)

# Add replicates to measurement
measurement_29.addReplicates(repeat1, enzmldoc)
measurement_29.addReplicates(repeat2, enzmldoc)

# Add the measurement to the EnzymeML document
meas_29_id = enzmldoc.addMeasurement(measurement_29)

print("Measurement created for two repeats at CTP = 0.0312 mM, Ppan = 0.046 mM and Cys = 1mM")

Measurement created for two repeats at CTP = 0.0312 mM, Ppan = 0.046 mM and Cys = 1mM


Create measurement for CTP = 0.0312 mM, PPan = 0.023 mM and Cys = 1mM

In [37]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1-CTP_0_0312.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2-CTP_0_0312.xlsx")

# Create the Measurement object for one run at CTP = 0.0312 mM
measurement_30 = Measurement(
    name="Run 30 - CTP = 0.0312 mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_30.addData(reactant_id=Ppan_id, init_conc=0.023, unit="mM")
measurement_30.addData(reactant_id=ctp_id, init_conc=0.0312, unit="mM")
measurement_30.addData(reactant_id=cys_id, init_conc=1, unit="mM")
measurement_30.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for CTP = 0.0312mM @ 0.023 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['F10'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for CTP = 0.0312 mM @ 0.023 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['F12'].tolist()
)

# Add replicates to measurement
measurement_30.addReplicates(repeat1, enzmldoc)
measurement_30.addReplicates(repeat2, enzmldoc)

# Add the measurement to the EnzymeML document
meas_30_id = enzmldoc.addMeasurement(measurement_30)

print("Measurement created for two repeats at CTP = 0.0312 mM, Ppan = 0.023 mM and Cys = 1mM")

Measurement created for two repeats at CTP = 0.0312 mM, Ppan = 0.023 mM and Cys = 1mM


Create NEW measurement for CTP = 3 mM, PPan = 0.37 mM and Cys = 1mM

In [38]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1_CTP_3mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2_CTP_3mM.xlsx")
df3 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run3_CTP_3mM.xlsx")

# Create the Measurement object for one run at CTP = 3 mM
measurement_31 = Measurement(
    name="Run 31 - CTP = 3 mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_31.addData(reactant_id=Ppan_id, init_conc=0.37, unit="mM")
measurement_31.addData(reactant_id=ctp_id, init_conc=3, unit="mM")
measurement_31.addData(reactant_id=cys_id, init_conc=1, unit="mM")
measurement_31.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for CTP = 3 mM @ 0.37 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['B2'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for CTP = 3 mM @ 0.37 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['B4'].tolist()
)

repeat3 = Replicate(
    id="repeat3 for CTP = 3 mM @ 0.37 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df3['Time_min'].tolist(),
    data=df3['B6'].tolist()
)

# Add replicates to measurement
measurement_31.addReplicates(repeat1, enzmldoc)
measurement_31.addReplicates(repeat2, enzmldoc)
measurement_31.addReplicates(repeat3, enzmldoc)

# Add the measurement to the EnzymeML document
meas_31_id = enzmldoc.addMeasurement(measurement_31)

print("Measurement created for three repeats at CTP = 3 mM, Ppan = 0.37 mM and Cys = 1mM")

Measurement created for three repeats at CTP = 3 mM, Ppan = 0.37 mM and Cys = 1mM


Create NEW measurement for CTP = 3 mM, PPan = 0.18 mM and Cys = 1mM

In [39]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1_CTP_3mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2_CTP_3mM.xlsx")
df3 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run3_CTP_3mM.xlsx")

# Create the Measurement object for one run at CTP = 3 mM
measurement_32 = Measurement(
    name="Run 32 - CTP = 3 mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_32.addData(reactant_id=Ppan_id, init_conc=0.18, unit="mM")
measurement_32.addData(reactant_id=ctp_id, init_conc=3, unit="mM")
measurement_32.addData(reactant_id=cys_id, init_conc=1, unit="mM")
measurement_32.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for CTP = 3 mM @ 0.18 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['C2'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for CTP = 3 mM @ 0.18 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['C4'].tolist()
)

repeat3 = Replicate(
    id="repeat3 for CTP = 3 mM @ 0.18 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df3['Time_min'].tolist(),
    data=df3['C6'].tolist()
)

# Add replicates to measurement
measurement_32.addReplicates(repeat1, enzmldoc)
measurement_32.addReplicates(repeat2, enzmldoc)
measurement_32.addReplicates(repeat3, enzmldoc)

# Add the measurement to the EnzymeML document
meas_32_id = enzmldoc.addMeasurement(measurement_32)

print("Measurement created for three repeats at CTP = 3 mM, Ppan = 0.18 mM and Cys = 1mM")

Measurement created for three repeats at CTP = 3 mM, Ppan = 0.18 mM and Cys = 1mM


Create NEW measurement for CTP = 3 mM, PPan = 0.093 mM and Cys = 1mM

In [40]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1_CTP_3mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2_CTP_3mM.xlsx")
df3 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run3_CTP_3mM.xlsx")

# Create the Measurement object for one run at CTP = 3 mM
measurement_33 = Measurement(
    name="Run 33 - CTP = 3 mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_33.addData(reactant_id=Ppan_id, init_conc=0.093, unit="mM")
measurement_33.addData(reactant_id=ctp_id, init_conc=3, unit="mM")
measurement_33.addData(reactant_id=cys_id, init_conc=1, unit="mM")
measurement_33.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for CTP = 3 mM @ 0.093 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['D2'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for CTP = 3 mM @ 0.093 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['D4'].tolist()
)

repeat3 = Replicate(
    id="repeat3 for CTP = 3 mM @ 0.093 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df3['Time_min'].tolist(),
    data=df3['D6'].tolist()
)

# Add replicates to measurement
measurement_33.addReplicates(repeat1, enzmldoc)
measurement_33.addReplicates(repeat2, enzmldoc)
measurement_33.addReplicates(repeat3, enzmldoc)

# Add the measurement to the EnzymeML document
meas_33_id = enzmldoc.addMeasurement(measurement_33)

print("Measurement created for three repeats at CTP = 3 mM, Ppan = 0.093 mM and Cys = 1mM")

Measurement created for three repeats at CTP = 3 mM, Ppan = 0.093 mM and Cys = 1mM


Create NEW measurement for CTP = 3 mM, PPan = 0.046 mM and Cys = 1mM

In [41]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1_CTP_3mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2_CTP_3mM.xlsx")
df3 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run3_CTP_3mM.xlsx")

# Create the Measurement object for one run at CTP = 3 mM
measurement_34 = Measurement(
    name="Run 34 - CTP = 3 mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_34.addData(reactant_id=Ppan_id, init_conc=0.046, unit="mM")
measurement_34.addData(reactant_id=ctp_id, init_conc=3, unit="mM")
measurement_34.addData(reactant_id=cys_id, init_conc=1, unit="mM")
measurement_34.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for CTP = 3 mM @ 0.046 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['E2'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for CTP = 3 mM @ 0.046 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['E4'].tolist()
)

repeat3 = Replicate(
    id="repeat3 for CTP = 3 mM @ 0.046 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df3['Time_min'].tolist(),
    data=df3['E6'].tolist()
)

# Add replicates to measurement
measurement_34.addReplicates(repeat1, enzmldoc)
measurement_34.addReplicates(repeat2, enzmldoc)
measurement_34.addReplicates(repeat3, enzmldoc)

# Add the measurement to the EnzymeML document
meas_34_id = enzmldoc.addMeasurement(measurement_34)

print("Measurement created for three repeats at CTP = 3 mM, Ppan = 0.046 mM and Cys = 1mM")

Measurement created for three repeats at CTP = 3 mM, Ppan = 0.046 mM and Cys = 1mM


Create measurement for CTP = 3 mM, PPan = 0.023 mM and Cys = 1mM

In [42]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1_CTP_3mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2_CTP_3mM.xlsx")
df3 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run3_CTP_3mM.xlsx")

# Create the Measurement object for one run at CTP = 3 mM
measurement_35 = Measurement(
    name="Run 35 - CTP = 3 mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_35.addData(reactant_id=Ppan_id, init_conc=0.023, unit="mM")
measurement_35.addData(reactant_id=ctp_id, init_conc=3, unit="mM")
measurement_35.addData(reactant_id=cys_id, init_conc=1, unit="mM")
measurement_35.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for CTP = 3 mM @ 0.023 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['F2'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for CTP = 3 mM @ 0.023 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['F4'].tolist()
)

repeat3 = Replicate(
    id="repeat3 for CTP = 3 mM @ 0.023 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df3['Time_min'].tolist(),
    data=df3['F6'].tolist()
)

# Add replicates to measurement
measurement_35.addReplicates(repeat1, enzmldoc)
measurement_35.addReplicates(repeat2, enzmldoc)
measurement_35.addReplicates(repeat3, enzmldoc)

# Add the measurement to the EnzymeML document
meas_35_id = enzmldoc.addMeasurement(measurement_35)

print("Measurement created for three repeats at CTP = 3 mM, Ppan = 0.023 mM and Cys = 1mM")

Measurement created for three repeats at CTP = 3 mM, Ppan = 0.023 mM and Cys = 1mM


Create NEW measurement for CTP = 2 mM, PPan = 0.37 mM and Cys = 1mM

In [43]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1_CTP_2mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2_CTP_2mM.xlsx")
df3 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run3_CTP_2mM.xlsx")

# Create the Measurement object for one run at CTP = 2 mM
measurement_36 = Measurement(
    name="Run 36 - CTP = 2 mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_36.addData(reactant_id=Ppan_id, init_conc=0.37, unit="mM")
measurement_36.addData(reactant_id=ctp_id, init_conc=2, unit="mM")
measurement_36.addData(reactant_id=cys_id, init_conc=1, unit="mM")
measurement_36.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for CTP = 2 mM @ 0.37 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['B8'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for CTP = 2 mM @ 0.37 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['B10'].tolist()
)

repeat3 = Replicate(
    id="repeat3 for CTP = 2 mM @ 0.37 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df3['Time_min'].tolist(),
    data=df3['B12'].tolist()
)

# Add replicates to measurement
measurement_36.addReplicates(repeat1, enzmldoc)
measurement_36.addReplicates(repeat2, enzmldoc)
measurement_36.addReplicates(repeat3, enzmldoc)

# Add the measurement to the EnzymeML document
meas_36_id = enzmldoc.addMeasurement(measurement_36)

print("Measurement created for three repeats at CTP = 2 mM, Ppan = 0.37 mM and Cys = 1mM")

Measurement created for three repeats at CTP = 2 mM, Ppan = 0.37 mM and Cys = 1mM


Create measurement for CTP = 2 mM, PPan = 0.18 mM and Cys = 1mM

In [44]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1_CTP_2mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2_CTP_2mM.xlsx")
df3 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run3_CTP_2mM.xlsx")

# Create the Measurement object for one run at CTP = 2 mM
measurement_37 = Measurement(
    name="Run 37 - CTP = 2 mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_37.addData(reactant_id=Ppan_id, init_conc=0.18, unit="mM")
measurement_37.addData(reactant_id=ctp_id, init_conc=2, unit="mM")
measurement_37.addData(reactant_id=cys_id, init_conc=1, unit="mM")
measurement_37.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for CTP = 2 mM @ 0.18 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['C8'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for CTP = 2 mM @ 0.18 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['C10'].tolist()
)

repeat3 = Replicate(
    id="repeat3 for CTP = 2 mM @ 0.18 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df3['Time_min'].tolist(),
    data=df3['C12'].tolist()
)

# Add replicates to measurement
measurement_37.addReplicates(repeat1, enzmldoc)
measurement_37.addReplicates(repeat2, enzmldoc)
measurement_37.addReplicates(repeat3, enzmldoc)

# Add the measurement to the EnzymeML document
meas_37_id = enzmldoc.addMeasurement(measurement_37)

print("Measurement created for three repeats at CTP = 2 mM, Ppan = 0.18 mM and Cys = 1mM")

Measurement created for three repeats at CTP = 2 mM, Ppan = 0.18 mM and Cys = 1mM


Create measurement for CTP = 2 mM, PPan = 0.095 mM and Cys = 1mM

In [45]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1_CTP_2mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2_CTP_2mM.xlsx")
df3 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run3_CTP_2mM.xlsx")

# Create the Measurement object for one run at CTP = 2 mM
measurement_38 = Measurement(
    name="Run 38 - CTP = 2 mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_38.addData(reactant_id=Ppan_id, init_conc=0.093, unit="mM")
measurement_38.addData(reactant_id=ctp_id, init_conc=2, unit="mM")
measurement_38.addData(reactant_id=cys_id, init_conc=1, unit="mM")
measurement_38.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for CTP = 2 mM @ 0.093 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['D8'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for CTP = 2 mM @ 0.093 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['D10'].tolist()
)

repeat3 = Replicate(
    id="repeat3 for CTP = 2 mM @ 0.093 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df3['Time_min'].tolist(),
    data=df3['D12'].tolist()
)

# Add replicates to measurement
measurement_38.addReplicates(repeat1, enzmldoc)
measurement_38.addReplicates(repeat2, enzmldoc)
measurement_38.addReplicates(repeat3, enzmldoc)

# Add the measurement to the EnzymeML document
meas_38_id = enzmldoc.addMeasurement(measurement_38)

print("Measurement created for three repeats at CTP = 2 mM, Ppan = 0.093 mM and Cys = 1mM")

Measurement created for three repeats at CTP = 2 mM, Ppan = 0.093 mM and Cys = 1mM


Create measurement for CTP = 2 mM, PPan = 0.046 mM and Cys = 1mM

In [46]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1_CTP_2mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2_CTP_2mM.xlsx")
df3 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run3_CTP_2mM.xlsx")

# Create the Measurement object for one run at CTP = 2 mM
measurement_39 = Measurement(
    name="Run 39 - CTP = 2 mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_39.addData(reactant_id=Ppan_id, init_conc=0.046, unit="mM")
measurement_39.addData(reactant_id=ctp_id, init_conc=2, unit="mM")
measurement_39.addData(reactant_id=cys_id, init_conc=1, unit="mM")
measurement_39.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for CTP = 2 mM @ 0.046 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['E8'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for CTP = 2 mM @ 0.046 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['E10'].tolist()
)

repeat3 = Replicate(
    id="repeat3 for CTP = 2 mM @ 0.046 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df3['Time_min'].tolist(),
    data=df3['E12'].tolist()
)

# Add replicates to measurement
measurement_39.addReplicates(repeat1, enzmldoc)
measurement_39.addReplicates(repeat2, enzmldoc)
measurement_39.addReplicates(repeat3, enzmldoc)

# Add the measurement to the EnzymeML document
meas_39_id = enzmldoc.addMeasurement(measurement_39)

print("Measurement created for three repeats at CTP = 2 mM, Ppan = 0.046 mM and Cys = 1mM")

Measurement created for three repeats at CTP = 2 mM, Ppan = 0.046 mM and Cys = 1mM


Create measurement for CTP = 2 mM, PPan = 0.023 mM and Cys = 1mM

In [47]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1_CTP_2mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2_CTP_2mM.xlsx")
df3 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run3_CTP_2mM.xlsx")

# Create the Measurement object for one run at CTP = 2 mM
measurement_40 = Measurement(
    name="Run 40 - CTP = 2 mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_40.addData(reactant_id=Ppan_id, init_conc=0.023, unit="mM")
measurement_40.addData(reactant_id=ctp_id, init_conc=2, unit="mM")
measurement_40.addData(reactant_id=cys_id, init_conc=1, unit="mM")
measurement_40.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for CTP = 2 mM @ 0.023 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['F8'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for CTP = 2 mM @ 0.023 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['F10'].tolist()
)

repeat3 = Replicate(
    id="repeat3 for CTP = 2 mM @ 0.023 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df3['Time_min'].tolist(),
    data=df3['F12'].tolist()
)

# Add replicates to measurement
measurement_40.addReplicates(repeat1, enzmldoc)
measurement_40.addReplicates(repeat2, enzmldoc)
measurement_40.addReplicates(repeat3, enzmldoc)

# Add the measurement to the EnzymeML document
meas_40_id = enzmldoc.addMeasurement(measurement_40)

print("Measurement created for three repeats at CTP = 2 mM, Ppan = 0.023 mM and Cys = 1mM")

Measurement created for three repeats at CTP = 2 mM, Ppan = 0.023 mM and Cys = 1mM


## Adding Cysteine measurements:

Create measurement for Cys =  0.5 mM, CTP = 1mM and Ppan = 0.37 mM

In [48]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1-Cys_0_5mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2-Cys_0_5mM.xlsx")
df3 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run3-Cys_0_5mM.xlsx")

# Create the Measurement object for one run at Cys = 0.5 mM
measurement_41 = Measurement(
    name="Run 41 - Cys = 0.5 mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_41.addData(reactant_id=Ppan_id, init_conc=0.37, unit="mM")
measurement_41.addData(reactant_id=ctp_id, init_conc=1, unit="mM")
measurement_41.addData(reactant_id=cys_id, init_conc=0.5, unit="mM")
measurement_41.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for Cys = 0.5 mM @ 0.37 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['B2'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for Cys = 0.5 mM @ 0.37 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['B12'].tolist()
)

repeat3 = Replicate(
    id="repeat3 for Cys = 0.5 mM @ 0.37 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df3['Time_min'].tolist(),
    data=df3['B2'].tolist()
)

# Add replicates to measurement
measurement_41.addReplicates(repeat1, enzmldoc)
measurement_41.addReplicates(repeat2, enzmldoc)
measurement_41.addReplicates(repeat3, enzmldoc)

# Add the measurement to the EnzymeML document
meas_41_id = enzmldoc.addMeasurement(measurement_41)

print("Measurement created for three repeats at Cys = 0.5 mM, Ppan = 0.37 mM and CTP = 1mM")

Measurement created for three repeats at Cys = 0.5 mM, Ppan = 0.37 mM and CTP = 1mM


Create measurement for Cys =  0.5 mM, CTP = 1mM and Ppan = 0.18 mM

In [49]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1-Cys_0_5mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2-Cys_0_5mM.xlsx")
df3 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run3-Cys_0_5mM.xlsx")

# Create the Measurement object for one run at Cys = 0.5 mM
measurement_42 = Measurement(
    name="Run 42 - Cys = 0.5 mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_42.addData(reactant_id=Ppan_id, init_conc=0.18, unit="mM")
measurement_42.addData(reactant_id=ctp_id, init_conc=1, unit="mM")
measurement_42.addData(reactant_id=cys_id, init_conc=0.5, unit="mM")
measurement_42.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for Cys = 0.5 mM @ 0.18 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['C2'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for Cys = 0.5 mM @ 0.18 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['C12'].tolist()
)

repeat3 = Replicate(
    id="repeat3 for Cys = 0.5 mM @ 0.18 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df3['Time_min'].tolist(),
    data=df3['C2'].tolist()
)

# Add replicates to measurement
measurement_42.addReplicates(repeat1, enzmldoc)
measurement_42.addReplicates(repeat2, enzmldoc)
measurement_42.addReplicates(repeat3, enzmldoc)

# Add the measurement to the EnzymeML document
meas_42_id = enzmldoc.addMeasurement(measurement_42)

print("Measurement created for three repeats at Cys = 0.5 mM, Ppan = 0.18 mM and CTP = 1mM")

Measurement created for three repeats at Cys = 0.5 mM, Ppan = 0.18 mM and CTP = 1mM


Create measurement for Cys =  0.5 mM, CTP = 1mM and Ppan = 0.093 mM

In [50]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1-Cys_0_5mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2-Cys_0_5mM.xlsx")
df3 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run3-Cys_0_5mM.xlsx")

# Create the Measurement object for one run at Cys = 0.5 mM
measurement_43 = Measurement(
    name="Run 43 - Cys = 0.5 mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_43.addData(reactant_id=Ppan_id, init_conc=0.093, unit="mM")
measurement_43.addData(reactant_id=ctp_id, init_conc=1, unit="mM")
measurement_43.addData(reactant_id=cys_id, init_conc=0.5, unit="mM")
measurement_43.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for Cys = 0.5 mM @ 0.093 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['D2'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for Cys = 0.5 mM @ 0.093 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['D12'].tolist()
)

repeat3 = Replicate(
    id="repeat3 for Cys = 0.5 mM @ 0.093 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df3['Time_min'].tolist(),
    data=df3['D2'].tolist()
)

# Add replicates to measurement
measurement_43.addReplicates(repeat1, enzmldoc)
measurement_43.addReplicates(repeat2, enzmldoc)
measurement_43.addReplicates(repeat3, enzmldoc)

# Add the measurement to the EnzymeML document
meas_43_id = enzmldoc.addMeasurement(measurement_43)

print("Measurement created for three repeats at Cys = 0.5 mM, Ppan = 0.093 mM and CTP = 1mM")

Measurement created for three repeats at Cys = 0.5 mM, Ppan = 0.093 mM and CTP = 1mM


Create measurement for Cys =  0.5 mM, CTP = 1mM and Ppan = 0.046 mM

In [51]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1-Cys_0_5mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2-Cys_0_5mM.xlsx")
df3 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run3-Cys_0_5mM.xlsx")

# Create the Measurement object for one run at Cys = 0.5 mM
measurement_44 = Measurement(
    name="Run 44 - Cys = 0.5 mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_44.addData(reactant_id=Ppan_id, init_conc=0.046, unit="mM")
measurement_44.addData(reactant_id=ctp_id, init_conc=1, unit="mM")
measurement_44.addData(reactant_id=cys_id, init_conc=0.5, unit="mM")
measurement_44.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for Cys = 0.5 mM @ 0.046 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['E2'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for Cys = 0.5 mM @ 0.046 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['E12'].tolist()
)

repeat3 = Replicate(
    id="repeat3 for Cys = 0.5 mM @ 0.046 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df3['Time_min'].tolist(),
    data=df3['E2'].tolist()
)

# Add replicates to measurement
measurement_44.addReplicates(repeat1, enzmldoc)
measurement_44.addReplicates(repeat2, enzmldoc)
measurement_44.addReplicates(repeat3, enzmldoc)

# Add the measurement to the EnzymeML document
meas_44_id = enzmldoc.addMeasurement(measurement_44)

print("Measurement created for three repeats at Cys = 0.5 mM, Ppan = 0.046 mM and CTP = 1mM")

Measurement created for three repeats at Cys = 0.5 mM, Ppan = 0.046 mM and CTP = 1mM


Create measurement for Cys =  0.5 mM, CTP = 1mM and Ppan = 0.023 mM

In [52]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1-Cys_0_5mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2-Cys_0_5mM.xlsx")
df3 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run3-Cys_0_5mM.xlsx")

# Create the Measurement object for one run at Cys = 0.5 mM
measurement_45 = Measurement(
    name="Run 45 - Cys = 0.5 mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_45.addData(reactant_id=Ppan_id, init_conc=0.023, unit="mM")
measurement_45.addData(reactant_id=ctp_id, init_conc=1, unit="mM")
measurement_45.addData(reactant_id=cys_id, init_conc=0.5, unit="mM")
measurement_45.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for Cys = 0.5 mM @ 0.023 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['F2'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for Cys = 0.5 mM @ 0.023 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['F12'].tolist()
)

repeat3 = Replicate(
    id="repeat3 for Cys = 0.5 mM @ 0.023 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df3['Time_min'].tolist(),
    data=df3['F2'].tolist()
)

# Add replicates to measurement
measurement_45.addReplicates(repeat1, enzmldoc)
measurement_45.addReplicates(repeat2, enzmldoc)
measurement_45.addReplicates(repeat3, enzmldoc)

# Add the measurement to the EnzymeML document
meas_45_id = enzmldoc.addMeasurement(measurement_45)

print("Measurement created for three repeats at Cys = 0.5 mM, Ppan = 0.023 mM and CTP = 1mM")

Measurement created for three repeats at Cys = 0.5 mM, Ppan = 0.023 mM and CTP = 1mM


New measurement for Cys: Create measurement for Cys =  0.1 mM, CTP = 1mM and Ppan = 0.37 mM

In [53]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1-Cys_0_1mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2-Cys_0_1mM.xlsx")
df3 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run3-Cys_0_1mM.xlsx")

# Create the Measurement object for one run at Cys = 0.1 mM
measurement_46 = Measurement(
    name="Run 46 - Cys = 0.1 mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_46.addData(reactant_id=Ppan_id, init_conc=0.37, unit="mM")
measurement_46.addData(reactant_id=ctp_id, init_conc=1, unit="mM")
measurement_46.addData(reactant_id=cys_id, init_conc=0.1, unit="mM")
measurement_46.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for Cys = 0.1 mM @ 0.37 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['B4'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for Cys = 0.1 mM @ 0.37 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['B4'].tolist()
)

repeat3 = Replicate(
    id="repeat3 for Cys = 0.1 mM @ 0.37 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df3['Time_min'].tolist(),
    data=df3['B4'].tolist()
)

# Add replicates to measurement
measurement_46.addReplicates(repeat1, enzmldoc)
measurement_46.addReplicates(repeat2, enzmldoc)
measurement_46.addReplicates(repeat3, enzmldoc)

# Add the measurement to the EnzymeML document
meas_46_id = enzmldoc.addMeasurement(measurement_46)

print("Measurement created for three repeats at Cys = 0.1 mM, Ppan = 0.37 mM and CTP = 1mM")

Measurement created for three repeats at Cys = 0.1 mM, Ppan = 0.37 mM and CTP = 1mM


 Create measurement for Cys = 0.1 mM, CTP = 1mM and Ppan = 0.18 mM

In [54]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1-Cys_0_1mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2-Cys_0_1mM.xlsx")
df3 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run3-Cys_0_1mM.xlsx")

# Create the Measurement object for one run at Cys = 0.1 mM
measurement_47 = Measurement(
    name="Run 47 - Cys = 0.1 mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_47.addData(reactant_id=Ppan_id, init_conc=0.18, unit="mM")
measurement_47.addData(reactant_id=ctp_id, init_conc=1, unit="mM")
measurement_47.addData(reactant_id=cys_id, init_conc=0.1, unit="mM")
measurement_47.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for Cys = 0.1 mM @ 0.18 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['C4'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for Cys = 0.1 mM @ 0.18 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['C4'].tolist()
)

repeat3 = Replicate(
    id="repeat3 for Cys = 0.1 mM @ 0.18 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df3['Time_min'].tolist(),
    data=df3['C4'].tolist()
)

# Add replicates to measurement
measurement_47.addReplicates(repeat1, enzmldoc)
measurement_47.addReplicates(repeat2, enzmldoc)
measurement_47.addReplicates(repeat3, enzmldoc)

# Add the measurement to the EnzymeML document
meas_47_id = enzmldoc.addMeasurement(measurement_47)

print("Measurement created for three repeats at Cys = 0.1 mM, Ppan = 0.18 mM and CTP = 1mM")

Measurement created for three repeats at Cys = 0.1 mM, Ppan = 0.18 mM and CTP = 1mM


Create measurement for Cys = 0.1 mM, CTP = 1mM and Ppan = 0.093 mM

In [55]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1-Cys_0_1mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2-Cys_0_1mM.xlsx")
df3 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run3-Cys_0_1mM.xlsx")

# Create the Measurement object for one run at Cys = 0.1 mM
measurement_48= Measurement(
    name="Run 48 - Cys = 0.1 mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_48.addData(reactant_id=Ppan_id, init_conc=0.093, unit="mM")
measurement_48.addData(reactant_id=ctp_id, init_conc=1, unit="mM")
measurement_48.addData(reactant_id=cys_id, init_conc=0.1, unit="mM")
measurement_48.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for Cys = 0.1 mM @ 0.093 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['D4'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for Cys = 0.1 mM @ 0.093 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['D4'].tolist()
)

repeat3 = Replicate(
    id="repeat3 for Cys = 0.1 mM @ 0.093 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df3['Time_min'].tolist(),
    data=df3['D4'].tolist()
)

# Add replicates to measurement
measurement_48.addReplicates(repeat1, enzmldoc)
measurement_48.addReplicates(repeat2, enzmldoc)
measurement_48.addReplicates(repeat3, enzmldoc)

# Add the measurement to the EnzymeML document
meas_48_id = enzmldoc.addMeasurement(measurement_48)

print("Measurement created for three repeats at Cys = 0.1 mM, Ppan = 0.093 mM and CTP = 1mM")

Measurement created for three repeats at Cys = 0.1 mM, Ppan = 0.093 mM and CTP = 1mM


Create measurement for Cys = 0.1 mM, CTP = 1mM and Ppan = 0.046 mM

In [56]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1-Cys_0_1mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2-Cys_0_1mM.xlsx")
df3 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run3-Cys_0_1mM.xlsx")

# Create the Measurement object for one run at Cys = 0.1 mM
measurement_49= Measurement(
    name="Run 49 - Cys = 0.1 mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_49.addData(reactant_id=Ppan_id, init_conc=0.046, unit="mM")
measurement_49.addData(reactant_id=ctp_id, init_conc=1, unit="mM")
measurement_49.addData(reactant_id=cys_id, init_conc=0.1, unit="mM")
measurement_49.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for Cys = 0.1 mM @ 0.046 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['E4'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for Cys = 0.1 mM @ 0.046 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['E4'].tolist()
)

repeat3 = Replicate(
    id="repeat3 for Cys = 0.1 mM @ 0.046 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df3['Time_min'].tolist(),
    data=df3['E4'].tolist()
)

# Add replicates to measurement
measurement_49.addReplicates(repeat1, enzmldoc)
measurement_49.addReplicates(repeat2, enzmldoc)
measurement_49.addReplicates(repeat3, enzmldoc)

# Add the measurement to the EnzymeML document
meas_49_id = enzmldoc.addMeasurement(measurement_49)

print("Measurement created for three repeats at Cys = 0.1 mM, Ppan = 0.046 mM and CTP = 1mM")

Measurement created for three repeats at Cys = 0.1 mM, Ppan = 0.046 mM and CTP = 1mM


Create measurement for Cys = 0.1 mM, CTP = 1mM and Ppan = 0.023 mM

In [57]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1-Cys_0_1mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2-Cys_0_1mM.xlsx")
df3 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run3-Cys_0_1mM.xlsx")

# Create the Measurement object for one run at Cys = 0.1 mM
measurement_50= Measurement(
    name="Run 50 - Cys = 0.1 mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_50.addData(reactant_id=Ppan_id, init_conc=0.023, unit="mM")
measurement_50.addData(reactant_id=ctp_id, init_conc=1, unit="mM")
measurement_50.addData(reactant_id=cys_id, init_conc=0.1, unit="mM")
measurement_50.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for Cys = 0.1 mM @ 0.023 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['F4'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for Cys = 0.1 mM @ 0.023 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['F4'].tolist()
)

repeat3 = Replicate(
    id="repeat3 for Cys = 0.1 mM @ 0.023 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df3['Time_min'].tolist(),
    data=df3['F4'].tolist()
)

# Add replicates to measurement
measurement_50.addReplicates(repeat1, enzmldoc)
measurement_50.addReplicates(repeat2, enzmldoc)
measurement_50.addReplicates(repeat3, enzmldoc)

# Add the measurement to the EnzymeML document
meas_50_id = enzmldoc.addMeasurement(measurement_50)

print("Measurement created for three repeats at Cys = 0.1 mM, Ppan = 0.023 mM and CTP = 1mM")

Measurement created for three repeats at Cys = 0.1 mM, Ppan = 0.023 mM and CTP = 1mM


Create new measurement for :  Cys = 0.05 mM, CTP = 1mM and Ppan = 0.37 mM

In [58]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1-Cys_0_05mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2-Cys_0_05mM.xlsx")
df3 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run3-Cys_0_05mM.xlsx")

# Create the Measurement object for one run at Cys = 0.05 mM
measurement_51= Measurement(
    name="Run 51 - Cys = 0.05 mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_51.addData(reactant_id=Ppan_id, init_conc=0.37, unit="mM")
measurement_51.addData(reactant_id=ctp_id, init_conc=1, unit="mM")
measurement_51.addData(reactant_id=cys_id, init_conc=0.05, unit="mM")
measurement_51.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for Cys = 0.05 mM @ 0.37 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['B6'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for Cys = 0.05 mM @ 0.37 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['B6'].tolist()
)

repeat3 = Replicate(
    id="repeat3 for Cys = 0.05 mM @ 0.37 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df3['Time_min'].tolist(),
    data=df3['B4'].tolist()
)

# Add replicates to measurement
measurement_51.addReplicates(repeat1, enzmldoc)
measurement_51.addReplicates(repeat2, enzmldoc)
measurement_51.addReplicates(repeat3, enzmldoc)

# Add the measurement to the EnzymeML document
meas_51_id = enzmldoc.addMeasurement(measurement_51)

print("Measurement created for three repeats at Cys = 0.05 mM, Ppan = 0.37 mM and CTP = 1mM")

Measurement created for three repeats at Cys = 0.05 mM, Ppan = 0.37 mM and CTP = 1mM


Create measurement for :  Cys = 0.05 mM, CTP = 1mM and Ppan = 0.18 mM

In [59]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1-Cys_0_05mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2-Cys_0_05mM.xlsx")
df3 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run3-Cys_0_05mM.xlsx")

# Create the Measurement object for one run at Cys = 0.05 mM
measurement_52= Measurement(
    name="Run 52 - Cys = 0.05 mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_52.addData(reactant_id=Ppan_id, init_conc=0.18, unit="mM")
measurement_52.addData(reactant_id=ctp_id, init_conc=1, unit="mM")
measurement_52.addData(reactant_id=cys_id, init_conc=0.05, unit="mM")
measurement_52.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for Cys = 0.05 mM @ 0.18 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['C6'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for Cys = 0.05 mM @ 0.18 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['C6'].tolist()
)

repeat3 = Replicate(
    id="repeat3 for Cys = 0.05 mM @ 0.18 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df3['Time_min'].tolist(),
    data=df3['C4'].tolist()
)

# Add replicates to measurement
measurement_52.addReplicates(repeat1, enzmldoc)
measurement_52.addReplicates(repeat2, enzmldoc)
measurement_52.addReplicates(repeat3, enzmldoc)

# Add the measurement to the EnzymeML document
meas_52_id = enzmldoc.addMeasurement(measurement_52)

print("Measurement created for three repeats at Cys = 0.05 mM, Ppan = 0.18 mM and CTP = 1mM")

Measurement created for three repeats at Cys = 0.05 mM, Ppan = 0.18 mM and CTP = 1mM


Create measurement for :  Cys = 0.05 mM, CTP = 1mM and Ppan = 0.093 mM

In [60]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1-Cys_0_05mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2-Cys_0_05mM.xlsx")
df3 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run3-Cys_0_05mM.xlsx")

# Create the Measurement object for one run at Cys = 0.05 mM
measurement_53= Measurement(
    name="Run 53 - Cys = 0.05 mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_53.addData(reactant_id=Ppan_id, init_conc=0.093, unit="mM")
measurement_53.addData(reactant_id=ctp_id, init_conc=1, unit="mM")
measurement_53.addData(reactant_id=cys_id, init_conc=0.05, unit="mM")
measurement_53.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for Cys = 0.05 mM @ 0.093 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['D6'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for Cys = 0.05 mM @ 0.093 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['D6'].tolist()
)

repeat3 = Replicate(
    id="repeat3 for Cys = 0.05 mM @ 0.093 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df3['Time_min'].tolist(),
    data=df3['D4'].tolist()
)

# Add replicates to measurement
measurement_53.addReplicates(repeat1, enzmldoc)
measurement_53.addReplicates(repeat2, enzmldoc)
measurement_53.addReplicates(repeat3, enzmldoc)

# Add the measurement to the EnzymeML document
meas_53_id = enzmldoc.addMeasurement(measurement_53)

print("Measurement created for three repeats at Cys = 0.05 mM, Ppan = 0.093 mM and CTP = 1mM")

Measurement created for three repeats at Cys = 0.05 mM, Ppan = 0.093 mM and CTP = 1mM


Create measurement for :  Cys = 0.05 mM, CTP = 1mM and Ppan = 0.046 mM

In [61]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1-Cys_0_05mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2-Cys_0_05mM.xlsx")
df3 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run3-Cys_0_05mM.xlsx")

# Create the Measurement object for one run at Cys = 0.05 mM
measurement_54= Measurement(
    name="Run 54 - Cys = 0.05 mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_54.addData(reactant_id=Ppan_id, init_conc=0.046, unit="mM")
measurement_54.addData(reactant_id=ctp_id, init_conc=1, unit="mM")
measurement_54.addData(reactant_id=cys_id, init_conc=0.05, unit="mM")
measurement_54.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for Cys = 0.05 mM @ 0.046 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['E6'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for Cys = 0.05 mM @ 0.046 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['E6'].tolist()
)

repeat3 = Replicate(
    id="repeat3 for Cys = 0.05 mM @ 0.046 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df3['Time_min'].tolist(),
    data=df3['E4'].tolist()
)

# Add replicates to measurement
measurement_54.addReplicates(repeat1, enzmldoc)
measurement_54.addReplicates(repeat2, enzmldoc)
measurement_54.addReplicates(repeat3, enzmldoc)

# Add the measurement to the EnzymeML document
meas_54_id = enzmldoc.addMeasurement(measurement_54)

print("Measurement created for three repeats at Cys = 0.05 mM, Ppan = 0.046 mM and CTP = 1mM")

Measurement created for three repeats at Cys = 0.05 mM, Ppan = 0.046 mM and CTP = 1mM


Create measurement for :  Cys = 0.05 mM, CTP = 1mM and Ppan = 0.023 mM

In [62]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1-Cys_0_05mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2-Cys_0_05mM.xlsx")
df3 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run3-Cys_0_05mM.xlsx")

# Create the Measurement object for one run at Cys = 0.05 mM
measurement_55= Measurement(
    name="Run 55 - Cys = 0.05 mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_55.addData(reactant_id=Ppan_id, init_conc=0.023, unit="mM")
measurement_55.addData(reactant_id=ctp_id, init_conc=1, unit="mM")
measurement_55.addData(reactant_id=cys_id, init_conc=0.05, unit="mM")
measurement_55.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for Cys = 0.05 mM @ 0.023 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['F6'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for Cys = 0.05 mM @ 0.023 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['F6'].tolist()
)

repeat3 = Replicate(
    id="repeat3 for Cys = 0.05 mM @ 0.023 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df3['Time_min'].tolist(),
    data=df3['F4'].tolist()
)

# Add replicates to measurement
measurement_55.addReplicates(repeat1, enzmldoc)
measurement_55.addReplicates(repeat2, enzmldoc)
measurement_55.addReplicates(repeat3, enzmldoc)

# Add the measurement to the EnzymeML document
meas_55_id = enzmldoc.addMeasurement(measurement_55)

print("Measurement created for three repeats at Cys = 0.05 mM, Ppan = 0.023 mM and CTP = 1mM")

Measurement created for three repeats at Cys = 0.05 mM, Ppan = 0.023 mM and CTP = 1mM


Create new measurement for :  Cys = 0.01 mM, CTP = 1mM and Ppan = 0.37 mM

In [63]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1-Cys_0_01mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2-Cys_0_01mM.xlsx")
df3 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run3-Cys_0_01mM.xlsx")

# Create the Measurement object for one run at Cys = 0.01 mM
measurement_56= Measurement(
    name="Run 56 - Cys = 0.01 mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_56.addData(reactant_id=Ppan_id, init_conc=0.37, unit="mM")
measurement_56.addData(reactant_id=ctp_id, init_conc=1, unit="mM")
measurement_56.addData(reactant_id=cys_id, init_conc=0.01, unit="mM")
measurement_56.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for Cys = 0.01 mM @ 0.37 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['B6'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for Cys = 0.01 mM @ 0.37 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['B6'].tolist()
)

repeat3 = Replicate(
    id="repeat3 for Cys = 0.01 mM @ 0.37 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df3['Time_min'].tolist(),
    data=df3['B8'].tolist()
)

# Add replicates to measurement
measurement_56.addReplicates(repeat1, enzmldoc)
measurement_56.addReplicates(repeat2, enzmldoc)
measurement_56.addReplicates(repeat3, enzmldoc)

# Add the measurement to the EnzymeML document
meas_56_id = enzmldoc.addMeasurement(measurement_56)

print("Measurement created for three repeats at Cys = 0.01 mM, Ppan = 0.37 mM and CTP = 1mM")

Measurement created for three repeats at Cys = 0.01 mM, Ppan = 0.37 mM and CTP = 1mM


Create measurement for :  Cys = 0.01 mM, CTP = 1mM and Ppan = 0.18 mM

In [64]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1-Cys_0_01mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2-Cys_0_01mM.xlsx")
df3 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run3-Cys_0_01mM.xlsx")

# Create the Measurement object for one run at Cys = 0.01 mM
measurement_57= Measurement(
    name="Run 57 - Cys = 0.01 mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_57.addData(reactant_id=Ppan_id, init_conc=0.18, unit="mM")
measurement_57.addData(reactant_id=ctp_id, init_conc=1, unit="mM")
measurement_57.addData(reactant_id=cys_id, init_conc=0.01, unit="mM")
measurement_57.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for Cys = 0.01 mM @ 0.18 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['C6'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for Cys = 0.01 mM @ 0.18 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['C6'].tolist()
)

repeat3 = Replicate(
    id="repeat3 for Cys = 0.01 mM @ 0.18 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df3['Time_min'].tolist(),
    data=df3['C8'].tolist()
)

# Add replicates to measurement
measurement_57.addReplicates(repeat1, enzmldoc)
measurement_57.addReplicates(repeat2, enzmldoc)
measurement_57.addReplicates(repeat3, enzmldoc)

# Add the measurement to the EnzymeML document
meas_57_id = enzmldoc.addMeasurement(measurement_57)

print("Measurement created for three repeats at Cys = 0.01 mM, Ppan = 0.18 mM and CTP = 1mM")

Measurement created for three repeats at Cys = 0.01 mM, Ppan = 0.18 mM and CTP = 1mM


Create measurement for :  Cys = 0.01 mM, CTP = 1mM and Ppan = 0.093 mM

In [65]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1-Cys_0_01mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2-Cys_0_01mM.xlsx")
df3 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run3-Cys_0_01mM.xlsx")

# Create the Measurement object for one run at Cys = 0.01 mM
measurement_58= Measurement(
    name="Run 58 - Cys = 0.01 mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_58.addData(reactant_id=Ppan_id, init_conc=0.093, unit="mM")
measurement_58.addData(reactant_id=ctp_id, init_conc=1, unit="mM")
measurement_58.addData(reactant_id=cys_id, init_conc=0.01, unit="mM")
measurement_58.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for Cys = 0.01 mM @ 0.093 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['D6'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for Cys = 0.01 mM @ 0.093 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['D6'].tolist()
)

repeat3 = Replicate(
    id="repeat3 for Cys = 0.01 mM @ 0.093 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df3['Time_min'].tolist(),
    data=df3['D8'].tolist()
)

# Add replicates to measurement
measurement_58.addReplicates(repeat1, enzmldoc)
measurement_58.addReplicates(repeat2, enzmldoc)
measurement_58.addReplicates(repeat3, enzmldoc)

# Add the measurement to the EnzymeML document
meas_58_id = enzmldoc.addMeasurement(measurement_58)

print("Measurement created for three repeats at Cys = 0.01 mM, Ppan = 0.093 mM and CTP = 1mM")

Measurement created for three repeats at Cys = 0.01 mM, Ppan = 0.093 mM and CTP = 1mM


Create measurement for : Cys = 0.01 mM, CTP = 1mM and Ppan = 0.046 mM

In [66]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1-Cys_0_01mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2-Cys_0_01mM.xlsx")
df3 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run3-Cys_0_01mM.xlsx")

# Create the Measurement object for one run at Cys = 0.01 mM
measurement_59= Measurement(
    name="Run 59 - Cys = 0.01 mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_59.addData(reactant_id=Ppan_id, init_conc=0.046, unit="mM")
measurement_59.addData(reactant_id=ctp_id, init_conc=1, unit="mM")
measurement_59.addData(reactant_id=cys_id, init_conc=0.01, unit="mM")
measurement_59.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for Cys = 0.01 mM @ 0.046 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['E6'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for Cys = 0.01 mM @ 0.046 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['E6'].tolist()
)

repeat3 = Replicate(
    id="repeat3 for Cys = 0.01 mM @ 0.046 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df3['Time_min'].tolist(),
    data=df3['E8'].tolist()
)

# Add replicates to measurement
measurement_59.addReplicates(repeat1, enzmldoc)
measurement_59.addReplicates(repeat2, enzmldoc)
measurement_59.addReplicates(repeat3, enzmldoc)

# Add the measurement to the EnzymeML document
meas_59_id = enzmldoc.addMeasurement(measurement_59)

print("Measurement created for three repeats at Cys = 0.01 mM, Ppan = 0.046 mM and CTP = 1mM")

Measurement created for three repeats at Cys = 0.01 mM, Ppan = 0.046 mM and CTP = 1mM


Create measurement for : Cys = 0.01 mM, CTP = 1mM and Ppan = 0.023 mM

In [67]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run1-Cys_0_01mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run2-Cys_0_01mM.xlsx")
df3 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\Bi Uni Uni Bi Ping Pong\Run3-Cys_0_01mM.xlsx")

# Create the Measurement object for one run at Cys = 0.01 mM
measurement_60= Measurement(
    name="Run 60 - Cys = 0.01 mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_60.addData(reactant_id=Ppan_id, init_conc=0.023, unit="mM")
measurement_60.addData(reactant_id=ctp_id, init_conc=1, unit="mM")
measurement_60.addData(reactant_id=cys_id, init_conc=0.01, unit="mM")
measurement_60.addData(protein_id=coabc_id, init_conc=0.05, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for Cys = 0.01 mM @ 0.023 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['F6'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for Cys = 0.01 mM @ 0.023 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['F6'].tolist()
)

repeat3 = Replicate(
    id="repeat3 for Cys = 0.01 mM @ 0.023 mM Ppan",
    species_id=ctp_id,
    data_unit="mM",
    time_unit="min",
    time=df3['Time_min'].tolist(),
    data=df3['F8'].tolist()
)

# Add replicates to measurement
measurement_60.addReplicates(repeat1, enzmldoc)
measurement_60.addReplicates(repeat2, enzmldoc)
measurement_60.addReplicates(repeat3, enzmldoc)

# Add the measurement to the EnzymeML document
meas_60_id = enzmldoc.addMeasurement(measurement_60)

print("Measurement created for three repeats at Cys = 0.01 mM, Ppan = 0.023 mM and CTP = 1mM")

Measurement created for three repeats at Cys = 0.01 mM, Ppan = 0.023 mM and CTP = 1mM


## Add Kinetic Model


In [68]:
# Create models py parsing equations for Sequential binding mechanism:

kinmod_PPCS = KineticModel.fromEquation("IMM_PPCS", 
                             "(vf*s0*s1*s3)/((K_ctp*K_ppan*K_cys)+(K_ppan*K_cys*s1)+(K_ctp*K_cys*s0)+(K_ctp*K_ppan*K_cys)+(s1*s0*s3))")


In [69]:
# Configure parameters for models

kinmod_PPCS._vf.update(value=7.58, unit="mM / min", lower=1e-6)
kinmod_PPCS._K_ctp.update(value=6.39, unit="mM", lower=1e-6)
kinmod_PPCS._K_ppan.update(value=0.014, unit="mM", lower=1e-6)
kinmod_PPCS._K_cys.update(value=0.0031, unit="mM", lower=1e-6)


In [70]:
# Add models to each reaction

reaction1.model = kinmod_PPCS

In [71]:
# Finally, write to EnzymeML
enzmldoc.toFile(".", "Enzyme ML doc")


Archive was written to .\Enzyme_ML_doc.omex



In [72]:
import numpy as np
from pyenzyme import EnzymeMLDocument

In [73]:
enzmldoc

Phosphopantothenoylcysteine synthetase (PPCS)
>>> Reactants
	ID: s0 	 Name: 4'-Phosphopantothenate
	ID: s1 	 Name: Cytidine triphosphate
	ID: s2 	 Name: Inorganic Pyrophospahte
	ID: s3 	 Name: L-cysteine
	ID: s4 	 Name: Cytidine monophosphate
	ID: s5 	 Name: 4'-Phosphopantothenoylcysteine
>>> Proteins
	ID: p0 	 Name: CoaBC
>>> Complexes
>>> Reactions
	ID: r0 	 Name: Phosphopantothenoylcysteine synthetase